<a href="https://colab.research.google.com/github/pdthuc/Project-Applied_Data_Science/blob/main/Group12_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KHOA HỌC DỮ LIỆU & ỨNG DỤNG
### GVHD: TRẦN TRUNG KIÊN
### THỰC HIỆN: Nhóm 12
### DANH SÁCH THÀNH VIÊN
  1. 1712667 - Lê Hữu Phúc
  2. 18120167 - Nguyễn Viết Dũng
  3. 18120579 - Đặng Minh Thọ
  4. 18120584 - Phạm Đình Thục

# 1. GIỚI THIỆU ĐỀ TÀI




- **Tên cuộc thi:** Recruit Restaurant Visitor Forecasting
- **Mô tả tổng quan:** Dự đoán nhà hàng sẽ nhận được bao nhiêu khách trong tương lai
- **Link cuộc thi:** https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting
- **Link `TOP 8` Solution:** https://github.com/MaxHalford/kaggle-recruit-restaurant


# 2. SƠ LƯỢC VỀ DỮ LIỆU


Dữ liệu đến từ hai trang web riêng biệt: 
  - Hot Pepper Gourmet (hpg): tại đây người dùng có thể tìm kiếm nhà hàng và đặt chỗ trực tuyến 
  - AirREGI / Restaurant Board (air): hệ thống kiểm soát đặt chỗ và tính tiền
  
Được thu thập từ năm 2016 đến tháng 4 năm 2017 
- AIR: 829 restaurants 
- HPG: 4690 restaurants

# 3. IMPORT CÁC THƯ VIỆN CẦN THIẾT

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/KHDL")
!ls


import pandas as pd
import numpy as np
import glob


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
data


# 4. KHÁM PHÁ DỮ LIỆU


---
## 4.1 File `air_visit_data.csv`

Trong file `air_visit_data.csv` liên quan về dữ liệu khách theo ngày: Trong file biểu diễn dữ liệu theo ngày (những ngày không có khách sẽ không được thu thập).

**Giải pháp:**

 -> Ta cần phải lấy mẫu theo ngày thay vì theo lượt truy cập.
 
  Do đó, những ngày không có khách thì lượt truy cập sẽ là 0. Có ích cho việc tính toán luân phiên dựa vào thời gian

In [ ]:
air_visit = pd.read_csv('data/air_visit_data.csv')

air_visit.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


* Sắp xếp dữ liệu theo ngày và thêm vào những này không được thu thập (số lượng khách bằng NaN)






In [ ]:
air_visit.index = pd.to_datetime(air_visit['visit_date'])
air_visit = air_visit.groupby('air_store_id').apply(lambda g: g['visitors'].resample('1d').sum()).reset_index()
air_visit.replace(0, np.nan, inplace=True)

air_visit.head()

,air_store_id,visit_date,visitors
0,air_00a91d42b08b08d9,2016-07-01,35.0
1,air_00a91d42b08b08d9,2016-07-02,9.0
2,air_00a91d42b08b08d9,2016-07-03,NaN
3,air_00a91d42b08b08d9,2016-07-04,20.0
4,air_00a91d42b08b08d9,2016-07-05,25.0


* Gán cờ `was_nil` cho những ngày không có khách

In [ ]:
air_visit['visit_date'] = air_visit['visit_date'].dt.strftime('%Y-%m-%d')
air_visit['was_nil'] = air_visit['visitors'].isnull()
air_visit['visitors'].fillna(0, inplace=True)

air_visit.head()

,air_store_id,visit_date,visitors,was_nil
0,air_00a91d42b08b08d9,2016-07-01,35.0,False
1,air_00a91d42b08b08d9,2016-07-02,9.0,False
2,air_00a91d42b08b08d9,2016-07-03,0.0,True
3,air_00a91d42b08b08d9,2016-07-04,20.0,False
4,air_00a91d42b08b08d9,2016-07-05,25.0,False


---
## 4.2 File `date_info.csv`


Trong file `date_info.csv` liên quan đến dữ liệu các ngày lễ.

Bao gồm: Thứ, ngày, tháng, năm và cờ để xác định có phải ngày lễ hay không?

**Giải pháp:** 
 Ta thêm hai tính năng cho biết ngày hôm trước và ngày hôm sau là ngày lễ hay không?

 -> Điều này giúp ta biết được ngày đó là ngày lễ đơn hay là kỳ nghỉ lễ dài ngày

In [ ]:
date_info = pd.read_csv('data/date_info.csv')

date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [ ]:
date_info.rename(columns={'holiday_flg': 'is_holiday', 'calendar_date': 'visit_date'}, inplace=True)
date_info['prev_day_is_holiday'] = date_info['is_holiday'].shift().fillna(0)
date_info['next_day_is_holiday'] = date_info['is_holiday'].shift(-1).fillna(0)

date_info.head()

,visit_date,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday
0,2016-01-01,Friday,1,0.0,1.0
1,2016-01-02,Saturday,1,1.0,1.0
2,2016-01-03,Sunday,1,1.0,0.0
3,2016-01-04,Monday,0,1.0,0.0
4,2016-01-05,Tuesday,0,0.0,0.0


---
## 4.3 File `air_store_info.csv`


Đối với thông tin của nhà hàng ở đây chúng ta sử dụng phiên bản xử lý trước thay vì phiên bản hiện tại của cuộc thi (vì nó chứa dữ liệu quan trọng của các trạm quan sát thời tiết).

Ta có thể truy cập dữ liệu này ở [đây](https://www.kaggle.com/huntermcgushion/rrv-weather-data).
Dữ liệu này chứa thêm các đặc trưng như: station_id (id của các trạm gần nhà hàng nhất), station_latitude (vĩ độ của trạm), station_longitude (kinh độ),…

In [ ]:
air_store_info = pd.read_csv('data/air_store_info.csv')

air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [ ]:
air_store_info = pd.read_csv('data/air_store_info_with_nearest_active_station.csv')

air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835


---
## 4.4 File `sample_submission.csv`


Tiếp theo chúng ta xử lý tập test, ở đây chúng ta sử dụng file `sample_submission.csv` để lấy dữ liệu test. Ta trích xuất id của nhà hàng và ngày tháng từ cột id. Vì id trong sample là chuỗi gồm id + visit_date.

**Giải pháp :**

-> Ta cần tách ra **id** riêng và **visit_date** riêng.

Cột `test_number` nhằm để các kết quả dự đoán (bài nộp) được sắp xếp theo đúng thứ tự.

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')
submission['air_store_id'] = submission['id'].str.slice(0, 20)
submission['visit_date'] = submission['id'].str.slice(21)
submission['is_test'] = True
submission['visitors'] = np.nan
submission['test_number'] = range(len(submission))

submission.head()

,id,visitors,air_store_id,visit_date,is_test,test_number
0,air_00a91d42b08b08d9_2017-04-23,NaN,air_00a91d42b08b08d9,2017-04-23,True,0
1,air_00a91d42b08b08d9_2017-04-24,NaN,air_00a91d42b08b08d9,2017-04-24,True,1
2,air_00a91d42b08b08d9_2017-04-25,NaN,air_00a91d42b08b08d9,2017-04-25,True,2
3,air_00a91d42b08b08d9_2017-04-26,NaN,air_00a91d42b08b08d9,2017-04-26,True,3
4,air_00a91d42b08b08d9_2017-04-27,NaN,air_00a91d42b08b08d9,2017-04-27,True,4


* Sau khi đã tách ra `air_store_id` và `visit_date`, xóa cột `id`

In [ ]:
data = pd.concat((air_visit, submission.drop('id', axis='columns')))

data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,test_number
0,air_00a91d42b08b08d9,2016-07-01,35.0,False,NaN,NaN
1,air_00a91d42b08b08d9,2016-07-02,9.0,False,NaN,NaN
2,air_00a91d42b08b08d9,2016-07-03,0.0,True,NaN,NaN
3,air_00a91d42b08b08d9,2016-07-04,20.0,False,NaN,NaN
4,air_00a91d42b08b08d9,2016-07-05,25.0,False,NaN,NaN


---
## 4.5 Merge dataset


Hợp nhất 2 bộ dữ liệu train và sample_submission sẽ giúp cho quá trình chọn lọc dữ liệu sau này thực hiện đơn giản hơn.


In [ ]:
data['is_test'].fillna(False, inplace=True)
data = pd.merge(left=data, right=date_info, on='visit_date', how='left')
data = pd.merge(left=data, right=air_store_info, on='air_store_id', how='left')
data['visitors'] = data['visitors'].astype(float)

data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,test_number,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
0,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
1,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
3,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
4,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906


--- 
## 4.6 Dữ liệu thời tiết

Do dữ liệu về thời tiết được lưu trữ ở nhiều file khác nhau, mỗi file ứng với 1 trạm thời tiết. 

**Giải pháp**

* Đầu tiên chúng ta nhập vào dữ liệu thời tiết.
* Liên kết `1663` file thời tiết với nhau vào bảng `weather`. Dữ liệu sau này sẽ chỉ xử lí thông qua bảng này.
* Trong bảng này có nhiều dữ liệu không tồn tại ~ NaN với nhiều lý do:    

  * 1 khu vực quanh năm nắng nóng sẽ không cần tới loại dữ liệu snowfall và tương tự.
  * 1 số trạm thời tiết nhỏ chỉ có 1 nhiệm vụ là đo lượng mưa và có những trạm lớn sẽ đo nhiều thông số thời tiết.
  * 1 số trạm ở khu vực vùng núi sẽ không thể đo đạc các chỉ số liên qua đến biển.

In [ ]:
import glob
weather_dfs = []

for path in glob.glob('data/1-1-16_5-31-17_Weather/*.csv'):
    weather_df = pd.read_csv(path)
    weather_df['station_id'] = path.split('\\')[-1].rstrip('.csv')
    weather_dfs.append(weather_df)


weather = pd.concat(weather_dfs, axis='rows')
weather.rename(columns={'calendar_date': 'visit_date'}, inplace=True)



In [ ]:
weather.head(20)

,visit_date,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,solar_radiation,deepest_snowfall,total_snowfall,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover,station_id
0,2016-01-01,1.1,11.7,-4.4,0.0,6.7,NaN,NaN,NaN,0.4,NaN,NaN,NaN,NaN,NaN,data/1-1-16_5-31-17_Weather/shizuoka__kawane-h...
1,2016-01-02,3.2,13.7,-4.7,0.0,6.7,NaN,NaN,NaN,0.9,NaN,NaN,NaN,NaN,NaN,data/1-1-16_5-31-17_Weather/shizuoka__kawane-h...
2,2016-01-03,5.2,16.9,-1.5,0.0,5.4,NaN,NaN,NaN,0.6,NaN,NaN,NaN,NaN,NaN,data/1-1-16_5-31-17_Weather/shizuoka__kawane-h...
3,2016-01-04,6.6,19.3,-2.0,0.0,6.8,NaN,NaN,NaN,1.1,NaN,NaN,NaN,NaN,NaN,data/1-1-16_5-31-17_Weather/shizuoka__kawane-h...
4,2016-01-05,7.4,17.9,-0.7,0.0,5.7,NaN,NaN,NaN,1.1,NaN,NaN,NaN,NaN,NaN,data/1-1-16_5-31-17_Weather/shizuoka__kawane-h...
5,2016-01-06,7.4,15.1,2.8,0.0,0.9,NaN,NaN,NaN,0.6,NaN,NaN,NaN,NaN,NaN,data/1-1-16_5-31-17_Weather/shizuoka__kawane-h...
6,2016-01-07,5.2,13.6,-0.6,0.0,6.8,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,data/1-1-16_5-31-17_Weather/shizuoka__kawane-h...
7,2016-01-08,2.6,10.5,-3.2,0.0,6.8,NaN,NaN,NaN,1.1,NaN,NaN,NaN,NaN,NaN,data/1-1-16_5-31-17_Weather/shizuoka__kawane-h...
8,2016-01-09,2.0,12.4,-3.4,0.0,6.9,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,data/1-1-16_5-31-17_Weather/shizuoka__kawane-h...
9,2016-01-10,2.8,13.0,-4.0,0.0,6.9,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,data/1-1-16_5-31-17_Weather/shizuoka__kawane-h...


---
## 4.7 Trích chọn dữ liệu thời tiết

Sau khi đã có thông tin về các chỉ số liên quan đến thời tiết, Ta có thể xem xét để trích chọn các giá trị thực sự ảnh hưởng đến số lượng khách hàng.
- Vì là số lượng khách hàng được tính trong 1 ngày nên các giá trị về nhiệt độ cao nhất hay thấp nhất trong 1 ngày ảnh hưởng ít đến số lượng khách.
- Đối với nhà hàng, thời gian mở cửa thường không thay đổi nên các chỉ số về mặt trời mọc hay lặn sẽ không ảnh hưởng đến các quyết định của nhà hàng.
- Ngoài ra còn có một số yếu tố chính, ảnh hưởng trực tiếp các yếu tố khác như nhiệt độ thường ảnh hưởng đến các yếu tố về tuyết nên cần lượt bỏ bớt.

Sau khi đã cân nhắc và xem xét, ta quyết định chọn ra các giá trị quan trọng đến số lượng khách:
- Lượng mưa
- Nhiệt độ
- (lượt bỏ các giá trị khác)

Tiếp theo, tiến hành tiền xử lý các giá trị thiếu bằng cách sử dụng giá trị trung bình (nhiệt độ và lượng mưa) hằng ngày trên **toàn cầu** để thay thế chúng 

In [ ]:
means = weather.groupby('visit_date')[['avg_temperature', 'precipitation']].mean().reset_index()
means.rename(columns={'avg_temperature': 'global_avg_temperature', 'precipitation': 'global_precipitation'}, inplace=True)
weather = pd.merge(left=weather, right=means, on='visit_date', how='left')
weather['avg_temperature'].fillna(weather['global_avg_temperature'], inplace=True)
weather['precipitation'].fillna(weather['global_precipitation'], inplace=True)

weather[['visit_date', 'avg_temperature', 'precipitation']].head()

,visit_date,avg_temperature,precipitation
0,2016-01-01,1.1,0.0
1,2016-01-02,3.2,0.0
2,2016-01-03,5.2,0.0
3,2016-01-04,6.6,0.0
4,2016-01-05,7.4,0.0


---
## 4.8 Kết quả thu được

Sau quá trình khám phá ta thu được bộ dữ liệu như sau:

In [ ]:
data['visit_date'] = pd.to_datetime(data['visit_date'])
data.index = data['visit_date']
#data.sort_values(['air_store_id', 'visit_date'], inplace=True)

data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,test_number,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
visit_date,,,,,,,,,,,,,,,,,,,,,
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906


# 5. TIỀN XỬ LÝ DỮ LIỆU


##5.1 Lượt bỏ outliers

Ta thực hiện lượt bỏ bớt các phần tử có giá trị lớn nhưng không thể hiện đúng phân bố của dữ liệu gây nhiễu cho quá trình huấn luyện(outlier).


Ví dụ: Số lượt truy cập vào dịp năm mới rõ ràng không đại diện cho các thời điểm còn lại trong năm.

Ta chọn hệ số Z là 2.4 để có khoản tin cậy khoản 97% và lượt bỏ khoản 3% số lượng giá trị outlier. Bạn có thể tham khảo tại [đây](https://www.mathsisfun.com/data/confidence-interval-calculator.html) để biết thêm về hệ số Z.


<div align="center">  <font size="5"> $\overline{x} \pm Z \frac{s}{\sqrt(n)}$ </font> </div>

Trong đó: 
- **$\overline{x}$**: là mean.
- **$Z$** là hệ số.
- **$s$** là là độ lệch chuẩn.
- **$n$** là số lượng mẫu quan sát.

|   | $Z$ |
|---|---|
| 80% | 1.282|
| 85% | 1.440 | 
| 90% | 1.645   |
| 95% | 1.960  | 
| 95% | 2.576 |
| 99.5% | 2.576 |
| 99.9% | 3.291 |

Những dòng có giá trị outlier sẻ được thay thế bằng max của những giá trị trong khoản tin cậy.

Tiếp theo ta tính log1p để transform lại dữ liệu, giúp dữ liệu phân bố tốt hơn, không bị lệch phải do các giá trị outlier chưa được lượt bỏ hết.

In [ ]:
def find_outliers(series):
    return (series - series.mean()) > 2.4 * series.std()


def cap_values(series):
    outliers = find_outliers(series)
    max_val = series[~outliers].max()
    series[outliers] = max_val
    return series


stores = data.groupby('air_store_id')
data['is_outlier'] = stores.apply(lambda g: find_outliers(g['visitors'])).values
data['visitors_capped'] = stores.apply(lambda g: cap_values(g['visitors'])).values
data['visitors_capped_log1p'] = np.log1p(data['visitors_capped'])

data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,test_number,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle,is_outlier,visitors_capped,visitors_capped_log1p
visit_date,,,,,,,,,,,,,,,,,,,,,,,,
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,35.0,3.583519
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,9.0,2.302585
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,0.0,0.000000
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,20.0,3.044522
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,25.0,3.258097


## 5.2 Trích xuất thêm các đặc trưng.

 Các đặc trưng cần thêm:
 - Ngày cuối tuần(`is_weekend`)
 - Ngày trong tháng(`day_of_month`)

Những đặt trưng này có thể được coi là đại diện cho thời điểm mọi người được trả tiền trong tháng, giả sử họ được trả tiền hàng tháng hay thường hay ăn chơi dịp cuối tuần.

In [ ]:
data['is_weekend'] = data['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
data['day_of_month'] = data['visit_date'].dt.day

## 5.3 Tính trọng số theo cấp số nhân cho số lượng khách hàng.

- Trọng số theo cấp số nhân - Exponentially weighted means (EWM) 

- Ta tính [EWM](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html) để tăng trọng số cho số lượng khách hàng của ngày hiện tại dựa trên những ngày trước đó, điều này nhằm mục đích nằm bắt được xu hướng của khách hàng.

- Để tính EWM cần biết tham số alpha. Ở đây ta dùng thư viện [optimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html) để tìm **alpha** tốt nhất cho giá trị EWM. Ta dùng độ lỗi **mean square error** để làm giá trị trả về cho hàm cần tối ưu. Ở đây ta cần tìm `min` của hàm f.

In [ ]:
from scipy import optimize


def calc_shifted_ewm(series, alpha, adjust=True):
    return series.shift().ewm(alpha=alpha, adjust=adjust).mean()


def find_best_signal(series, adjust=False, eps=10e-5):
    
    def f(alpha):
        shifted_ewm = calc_shifted_ewm(series=series, alpha=min(max(alpha, 0), 1), adjust=adjust)
        corr = np.mean(np.power(series - shifted_ewm, 2))
        return corr
     
    res = optimize.differential_evolution(func=f, bounds=[(0 + eps, 1 - eps)])
    
    return calc_shifted_ewm(series=series, alpha=res['x'][0], adjust=adjust)


roll = data.groupby(['air_store_id', 'day_of_week']).apply(lambda g: find_best_signal(g['visitors_capped']))
data['optimized_ewm_by_air_store_id_&_day_of_week'] = roll.sort_index(level=['air_store_id', 'visit_date']).values

roll = data.groupby(['air_store_id', 'is_weekend']).apply(lambda g: find_best_signal(g['visitors_capped']))
data['optimized_ewm_by_air_store_id_&_is_weekend'] = roll.sort_index(level=['air_store_id', 'visit_date']).values

roll = data.groupby(['air_store_id', 'day_of_week']).apply(lambda g: find_best_signal(g['visitors_capped_log1p']))
data['optimized_ewm_log1p_by_air_store_id_&_day_of_week'] = roll.sort_index(level=['air_store_id', 'visit_date']).values

roll = data.groupby(['air_store_id', 'is_weekend']).apply(lambda g: find_best_signal(g['visitors_capped_log1p']))
data['optimized_ewm_log1p_by_air_store_id_&_is_weekend'] = roll.sort_index(level=['air_store_id', 'visit_date']).values

data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,test_number,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle,is_outlier,visitors_capped,visitors_capped_log1p,is_weekend,day_of_month,optimized_ewm_by_air_store_id_&_day_of_week,optimized_ewm_by_air_store_id_&_is_weekend,optimized_ewm_log1p_by_air_store_id_&_day_of_week,optimized_ewm_log1p_by_air_store_id_&_is_weekend
visit_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,35.0,3.583519,0,1,NaN,NaN,NaN,NaN
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,9.0,2.302585,1,2,NaN,NaN,NaN,NaN
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,0.0,0.000000,1,3,NaN,9.000000,NaN,2.302585
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,20.0,3.044522,0,4,NaN,35.000000,NaN,3.583519
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,25.0,3.258097,0,5,NaN,33.047459,NaN,3.454852


## 5.4 Tính toán các đặc trưng.

Tính các đặc trưng như:
- Trung bình (mean)
- Trung vị (median)
- Độ lệch chuẩn (standard deviation)
- Số lượng giá trị
- Tối thiểu (min)
- Tối đa (max)
- Giá trị EWM.

In [ ]:
def extract_precedent_statistics(df, on, group_by):

    df.index.name = None
    
    df.sort_values(group_by + ['visit_date'], inplace=True)
    
    groups = df.groupby(group_by, sort=False)
    
    stats = {
        'mean': [],
        'median': [],
        'std': [],
        'count': [],
        'max': [],
        'min': []
    }
    
    exp_alphas = [0.1, 0.25, 0.3, 0.5, 0.75]
    stats.update({'exp_{}_mean'.format(alpha): [] for alpha in exp_alphas})
    
    for _, group in groups:
        
        shift = group[on].shift()
        roll = shift.rolling(window=len(group), min_periods=1)
        
        stats['mean'].extend(roll.mean())
        stats['median'].extend(roll.median())
        stats['std'].extend(roll.std())
        stats['count'].extend(roll.count())
        stats['max'].extend(roll.max())
        stats['min'].extend(roll.min())
        
        for alpha in exp_alphas:
            exp = shift.ewm(alpha=alpha, adjust=False)
            stats['exp_{}_mean'.format(alpha)].extend(exp.mean())
    
    suffix = '_&_'.join(group_by)
    
    for stat_name, values in stats.items():
        df['{}_{}_by_{}'.format(on, stat_name, suffix)] = values


extract_precedent_statistics(
    df=data,
    on='visitors_capped',
    group_by=['air_store_id', 'day_of_week']
)

extract_precedent_statistics(
    df=data,
    on='visitors_capped',
    group_by=['air_store_id', 'is_weekend']
)

extract_precedent_statistics(
    df=data,
    on='visitors_capped',
    group_by=['air_store_id']
)

extract_precedent_statistics(
    df=data,
    on='visitors_capped_log1p',
    group_by=['air_store_id', 'day_of_week']
)

extract_precedent_statistics(
    df=data,
    on='visitors_capped_log1p',
    group_by=['air_store_id', 'is_weekend']
)

extract_precedent_statistics(
    df=data,
    on='visitors_capped_log1p',
    group_by=['air_store_id']
)

data.sort_values(['air_store_id', 'visit_date']).head()


,air_store_id,visit_date,visitors,was_nil,is_test,test_number,day_of_week,is_holiday,prev_day_is_holiday,next_day_is_holiday,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle,is_outlier,visitors_capped,visitors_capped_log1p,is_weekend,day_of_month,optimized_ewm_by_air_store_id_&_day_of_week,optimized_ewm_by_air_store_id_&_is_weekend,optimized_ewm_log1p_by_air_store_id_&_day_of_week,optimized_ewm_log1p_by_air_store_id_&_is_weekend,visitors_capped_mean_by_air_store_id_&_day_of_week,visitors_capped_median_by_air_store_id_&_day_of_week,visitors_capped_std_by_air_store_id_&_day_of_week,visitors_capped_count_by_air_store_id_&_day_of_week,visitors_capped_max_by_air_store_id_&_day_of_week,visitors_capped_min_by_air_store_id_&_day_of_week,visitors_capped_exp_0.1_mean_by_air_store_id_&_day_of_week,visitors_capped_exp_0.25_mean_by_air_store_id_&_day_of_week,visitors_capped_exp_0.3_mean_by_air_store_id_&_day_of_week,visitors_capped_exp_0.5_mean_by_air_store_id_&_day_of_week,...,visitors_capped_max_by_air_store_id,visitors_capped_min_by_air_store_id,visitors_capped_exp_0.1_mean_by_air_store_id,visitors_capped_exp_0.25_mean_by_air_store_id,visitors_capped_exp_0.3_mean_by_air_store_id,visitors_capped_exp_0.5_mean_by_air_store_id,visitors_capped_exp_0.75_mean_by_air_store_id,visitors_capped_log1p_mean_by_air_store_id_&_day_of_week,visitors_capped_log1p_median_by_air_store_id_&_day_of_week,visitors_capped_log1p_std_by_air_store_id_&_day_of_week,visitors_capped_log1p_count_by_air_store_id_&_day_of_week,visitors_capped_log1p_max_by_air_store_id_&_day_of_week,visitors_capped_log1p_min_by_air_store_id_&_day_of_week,visitors_capped_log1p_exp_0.1_mean_by_air_store_id_&_day_of_week,visitors_capped_log1p_exp_0.25_mean_by_air_store_id_&_day_of_week,visitors_capped_log1p_exp_0.3_mean_by_air_store_id_&_day_of_week,visitors_capped_log1p_exp_0.5_mean_by_air_store_id_&_day_of_week,visitors_capped_log1p_exp_0.75_mean_by_air_store_id_&_day_of_week,visitors_capped_log1p_mean_by_air_store_id_&_is_weekend,visitors_capped_log1p_median_by_air_store_id_&_is_weekend,visitors_capped_log1p_std_by_air_store_id_&_is_weekend,visitors_capped_log1p_count_by_air_store_id_&_is_weekend,visitors_capped_log1p_max_by_air_store_id_&_is_weekend,visitors_capped_log1p_min_by_air_store_id_&_is_weekend,visitors_capped_log1p_exp_0.1_mean_by_air_store_id_&_is_weekend,visitors_capped_log1p_exp_0.25_mean_by_air_store_id_&_is_weekend,visitors_capped_log1p_exp_0.3_mean_by_air_store_id_&_is_weekend,visitors_capped_log1p_exp_0.5_mean_by_air_store_id_&_is_weekend,visitors_capped_log1p_exp_0.75_mean_by_air_store_id_&_is_weekend,visitors_capped_log1p_mean_by_air_store_id,visitors_capped_log1p_median_by_air_store_id,visitors_capped_log1p_std_by_air_store_id,visitors_capped_log1p_count_by_air_store_id,visitors_capped_log1p_max_by_air_store_id,visitors_capped_log1p_min_by_air_store_id,visitors_capped_log1p_exp_0.1_mean_by_air_store_id,visitors_capped_log1p_exp_0.25_mean_by_air_store_id,visitors_capped_log1p_exp_0.3_mean_by_air_store_id,visitors_capped_log1p_exp_0.5_mean_by_air_store_id,visitors_capped_log1p_exp_0.75_mean_by_air_store_id
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,35.0,3.583519,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906,False,9.0,2.302585,1,2,NaN,NaN,N

## 5.5 Mã hoá dữ liệu

In [ ]:
data = pd.get_dummies(data, columns=['day_of_week', 'air_genre_name'])

# 6. XÂY DỰNG MÔ HÌNH


# 7. ĐÁNH GIÁ MÔ HÌNH 